# FDA Drug Repurposing - Join and Load to CAS

0) run this AFTER DrugBank & FDA Clinical Trials python processes

1) set up envrionment
- if download directories match Box - this program should run without modification
- to enable CAS table load:
    - edit Cell 3 "casenabled" to true
    - modify CAS server to your address
    - modify CAS ID to your ID
    - and add your password

2) Read in dimensions for drug repositioning
<br><br>
- DrugBank: Drug (pre processed by DrugBank_drug_target_relationship_v2.ipynb)
dbSmallM_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-slim.tsv', sep='\t')
dbIndication_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-indication.tsv', sep='\t')
- DrugBank: Drug/Protein
dbProteins_df = pd.read_csv(DBPath + 'DrugBank_CSVs/proteins.tsv', sep='\t')
- DrugBank: synonyms
fObj = open(DBPath+'DrugBank_CSVs/aliases.json',)

<br><br>
- ClinicalTrials: COVID Truth (pre processed by FDA_Read_ClinicalTrialsTruth.ipynb)
Truth_df = pd.read_csv(CTPath + 'ProposedTruth.csv')
truthFinal_df = pd.read_csv(CASPublicPath + 'truthfinal.csv') - from PUBLIC CASLIB

<br><br>
- UniProt: Protein synonym IDs for joining Drugbank, String and VirusHost proteins<br>
with open(UniProtPath + 'HUMAN_9606_idmapping.dat') as f:
    row_count = sum(1 for row in f)
display(row_count)

    source: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/by_organism/
<br><br>
- VirusHost: SARS Related Virus to Host Protein
    - VirusHostProt_df = pd.read_csv(VHProtPath + 'HCoV-associated host proteins.csv')
- String: Protein-Protein
    - stringPP_df = pd.read_csv(STRINGPath + '9606.protein.links.v11.0.txt', sep=' ')
<br>

3) write out base tables to CASLIB + CSVs
- UniProt Lookups
    - UniProtGeneIDs_df.to_csv(CSVOut+"/"+"UniProtGeneIDs.csv")
    - UniProtGeneIDs_ct = s.upload_file(data=UniProtGeneIDs_df, casout=dict(name='UniProtGeneIDs', caslib='public', replication=0, promote=True))
    
    - UniProtSTRINGIDs_df.to_csv(CSVOut+"/"+"UniProtSTRINGIDs.csv")
    - UniProtSTRINGIDs_ct = s.upload_file(data=UniProtSTRINGIDs_df, casout=dict(name='UniProtSTRINGIDs', caslib='public', replication=0, promote=True))
    
    - UniProtGeneNames_df.to_csv(CSVOut+"/"+"UniProtGeneNames.csv")
    - UniProtGeneNames_ct = s.upload_file(data=UniProtGeneNames_df, casout=dict(name='UniProtGeneNames', caslib='public', replication=0, promote=True))

- Truth Sources
    - Truth_df.to_csv(CSVOut+"/"+"Truth.csv")
    - Truth_ct = s.upload_file(data=Truth_df, casout=dict(name='Truth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))

    - truthMatch2_df.to_csv(CSVOut+"/"+"newTruth.csv")
    - newTruth_ct = s.upload_file(data=truthMatch2_df, casout=dict(name='newTruth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))

- String mapped to UniProt so it will match DrugBank Drug-Protein IDs
    - stringPP_ct = s.upload_file(data=stringPP_df, casout=dict(name='stringPP', caslib='public', replication=0, promote=True, label="STRING Protein Protein interactions NOTE Missing UniProt Matches"))
    - stringPP_df.to_csv(CSVOut+"/"+"stringPP.csv")

- Virus Host Protein mapped to UniProt so it will match DrugBank Drug-Protein IDs
    - VirusHostProt_df.to_csv(CSVOut+"/"+"VirusHostProt.csv")
    - VirusHostProt_ct = s.upload_file(data=VirusHostProt_df, casout=dict(name='VirusHostProt', caslib='public', replication=0, promote=True, label="Virus Host Protein Relationships NOTE: Many Proteins had 1+ UniProt IDs"))

- DrugBank_IDs mapped to both "truth" tables
    - x.to_csv(CSVOut+"/"+"dbSmallM_Truth.csv")
    - dbSmallM_ct = s.upload_file(data=x, casout=dict(name='dbSmallM_Truth', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molecule drugs (NOTE additional attributes available plus stings need to be parsed)"))

- Drugbank Drug-Proteins
    - dbProteins_df.to_csv(CSVOut+"/"+"dbProteins.csv")
    - dbProteins_ct = s.upload_file(data=dbProteins_df, casout=dict(name='dbProteins', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molcule related proteins (NOTE not all captured by XML parser yet)"))


## 1) setup environment

In [1]:
# load packages

import pandas as pd
import numpy as np
import os

import time
import getpass


In [2]:
# files / paths
print(os.getcwd())
FDAPath = os.getcwd() + '/../data/source/'

CTPath = FDAPath + 'ClinicalTrials/'
DBPath = FDAPath + "DrugBank/"
STRINGPath = FDAPath + 'STRING/'
VHProtPath = FDAPath + 'VirusHostProteins/'
UniProtPath = FDAPath + 'UniProt/'

CASPublicPath = '/opt/sas/viya/config/data/cas/default/public/'   # truthfinal.csv exported from cas
CSVOut = FDAPath + '../DrugRepositioning'

print(os.getcwd())
os.chdir(FDAPath)
print(os.getcwd())




/home/sasdemo05/Python
/home/sasdemo05/Python
/home/sasdemo05/data/source


In [3]:
# link to CAS
# https://developer.sas.com/guides/python-swat.html

casenabled = False
casServerURL = "pdcesx06019.exnet.sas.com"

casPort = 5570
sasid = 'sasdemo'
saspw = 'xxxxx'
if casenabled == True:
    saspw = '<set your pw here>' #getpass.getpass() 
    import swat
    s = swat.CAS(casServerURL, casPort, sasid, saspw)
    #s = swat.CAS("pdcesx15065.exnet.sas.com", 5570, 'sasdemo', authinfo='/home/sasdemo/.authinfo')
    print("CAS is enabled")
else:
    print("No CAS")

CAS is enabled


In [4]:
if casenabled == True:
    s.serverstatus()


NOTE: Grid node action status report: 1 nodes, 9 total actions executed.


## 2) read base tables

In [5]:
# DrugBank (this is a database with multiple tables)
#  pre-processed in drug_target_relationship_DrugBank_rkc.ipynb
#  note: needs more work (as of 2021 02 18)

# drug bank - same info as "Slim" but all IDs instead of just small molecule
#db_df = pd.read_csv(DBPath + 'drugbank.tsv', sep='\t')
#display(db_df.head())

# drug bank - Small Molecule IDs only
dbSmallM_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-slim.tsv', sep='\t')
dbIndication_df = pd.read_csv(DBPath + 'DrugBank_CSVs/drugbank-indication.tsv', sep='\t')
display(dbSmallM_df.shape)
dbSmallM_df = dbSmallM_df.merge(dbIndication_df[['drugbank_id', 'indication']], on='drugbank_id', how='left')
display(dbSmallM_df.shape)
display(dbSmallM_df.head())
del dbIndication_df

dbProteins_df = pd.read_csv(DBPath + 'DrugBank_CSVs/proteins.tsv', sep='\t')
dbProteins_df = dbProteins_df.merge(dbSmallM_df[['drugbank_id']], on='drugbank_id', how='inner')
display(dbProteins_df.shape)
display(dbProteins_df.head())


(11414, 9)

(11414, 10)

,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi,description,indication
0,DB00006,Bivalirudin,small molecule,approved|investigational,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,Bivalirudin is a synthetic 20 residue peptide ...,For treatment of heparin-induced thrombocytope...
1,DB00007,Leuprolide,small molecule,approved|investigational,L02AE51|L02AE02,Adrenal Cortex Hormones|Agents Causing Muscle ...,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,Leuprolide is a synthetic 9-residue peptide an...,Leuprolide is indicated for the palliative tre...
2,DB00014,Goserelin,small molecule,approved,L02AE03,"Adrenal Cortex Hormones|Amino Acids, Peptides,...",BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,"Goserelin is a synthetic hormone. In men, it s...",Goserelin is indicated for:\r\n\r\n- Use in co...
3,DB00027,Gramicidin D,small molecule,approved,R02AB30,"Amino Acids, Peptides, and Proteins|Anti-Bacte...",NDAYQJDHGXTBJL-MWWSRJDJSA-N,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,Gramcidin D is a heterogeneous mixture of thre...,"For treatment of skin lesions, surface wounds ..."
4,DB00035,Desmopressin,small molecule,approved,H01BA02,"Agents that produce hypertension|Amino Acids, ...",NFLWUMRGJYTJIN-PNIOQBSNSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,"Desmopressin (dDAVP), a synthetic analogue of ...",- Indicated for the treatment of nocturia due ...


(25940, 4)

,drugbank_id,category,uniprot_id,organism
0,DB00006,target,P00734,Humans
1,DB00006,enzyme,P05164,Humans
2,DB00007,target,P30968,Humans
3,DB00014,target,P22888,Humans
4,DB00014,target,P30968,Humans


In [6]:
# FDA Truth - list of DrugBank Drug IDs that should be considered the "right" answers
#  pre-processed in FDA_Read_ClinicalTrialsTruth.ipynb
#  note: needs more work (as of 2021 02 18)
Truth_df = pd.read_csv(CTPath + 'ProposedTruth.csv')
display(Truth_df.head())

Truth_df.rename(columns = {'iDBID':'drugbank_id'}, inplace = True)
Truth_df['truth'] = 1
Truth_df = Truth_df[['drugbank_id', 'truth', 'iDBIDCount', 'iName', 'inputRowCount', 'inputRowIDs']]
display(Truth_df.shape)
display(Truth_df.head())


,Unnamed: 0,iName,inputRowCount,inputRowIDs,iDBIDCount,iDBID
0,83,AMY-101,1,1891,1.0,DB14803
1,108,AT-001,1,308,1.0,DB15121
2,124,AZD1222,3,[ 14 19 231],1.0,DB15656
3,129,Abatacept,3,[309 583 607],1.0,DB01281
4,133,Abivertinib,1,1352,1.0,DB15327


(315, 6)

,drugbank_id,truth,iDBIDCount,iName,inputRowCount,inputRowIDs
0,DB14803,1,1.0,AMY-101,1,1891
1,DB15121,1,1.0,AT-001,1,308
2,DB15656,1,1.0,AZD1222,3,[ 14 19 231]
3,DB01281,1,1.0,Abatacept,3,[309 583 607]
4,DB15327,1,1.0,Abivertinib,1,1352


In [7]:
# Truth Final - from (?) inn CAS Public folder
# probably could read directly from PUBLIC caslib

#####  NOTE !!!   This came AFTER the above table - was imported to CAS 
#####               a new source was appended
#####               and now this replaces the above in the process

truthFinal_df = pd.read_csv(CASPublicPath + 'truthfinal.csv')
display(truthFinal_df.shape)
display(truthFinal_df.head(20))


(619, 2)

,NewDrug,Trail_Drug
0,0,Amlodipine
1,0,AMY-101
2,0,Angiotensin 1-7
3,1,Antibiotic nos
4,1,Auxora
5,1,Auxora
6,0,AZD1222
7,0,Berberine
8,1,Beta blocker
9,1,Calcineurin inhibitors nos


In [8]:
# match CASLIB truthFinal to Drugbank synonyms json
# (extracted code from FDA_Read_ClincalTrialsTruth.ipynb)


import json

fObj = open(DBPath+'DrugBank_CSVs/aliases.json',)
DBAlias = json.load(fObj)
fObj.close()

print(DBAlias.__class__)
display(DBAlias['DB12466'])
display(DBAlias["DB15327"])

# search dict in DBAlias format to return ID if element of an array matches
def searchDBAlias(byVal):
    keysList = []
    itemsList = DBAlias.items()
    for item in itemsList:
        if byVal in item[1]:
            keysList.append(item[0])
    return keysList

# this could be sped up if it becomes an issue
start = time.time()
truthFinal_df['Trail_Drug_DBID'] = truthFinal_df['Trail_Drug'].apply(searchDBAlias)
lapse = time.time() - start 
print("lapse time to match: ", lapse)


<class 'dict'>


['Avigan', 'Fapilavir', 'Favilavir', 'Favipiravir']

['Abivertinib']

lapse time to match:  4.934699773788452


In [9]:
# count Drug Bank ID matches
truthFinal_df['iDBIDCount'] = truthFinal_df['Trail_Drug_DBID'].astype('str').str.count("'")/2 
display(truthFinal_df.shape)
display(truthFinal_df.head(20))


(619, 4)

,NewDrug,Trail_Drug,Trail_Drug_DBID,iDBIDCount
0,0,Amlodipine,[DB00381],1.0
1,0,AMY-101,[DB14803],1.0
2,0,Angiotensin 1-7,[DB11720],1.0
3,1,Antibiotic nos,[],0.0
4,1,Auxora,[],0.0
5,1,Auxora,[],0.0
6,0,AZD1222,[DB15656],1.0
7,0,Berberine,[DB04115],1.0
8,1,Beta blocker,[],0.0
9,1,Calcineurin inhibitors nos,[],0.0


In [10]:
display(truthFinal_df.shape)
display(truthFinal_df.head(20))

#display(truthFinal_df.Trail_Drug_DBID.apply(len)==0)
#display(truthFinal_df.NewDrug==1)
#display(np.logical_and(truthFinal_df.NewDrug==1, truthFinal_df.Trail_Drug_DBID.apply(len)==0))

#display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.Trail_Drug_DBID.apply(len)!=0))].shape)
#display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.Trail_Drug_DBID.apply(len)!=0))].head(50))

display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.iDBIDCount!=0))].shape)
display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==0), (truthFinal_df.iDBIDCount!=0))].head(50))

display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount!=0))].shape)
display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount!=0))].head(50))

display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount==0))].shape)
display(truthFinal_df[np.logical_and((truthFinal_df.NewDrug==1), (truthFinal_df.iDBIDCount==0))].head(200))


(619, 4)

,NewDrug,Trail_Drug,Trail_Drug_DBID,iDBIDCount
0,0,Amlodipine,[DB00381],1.0
1,0,AMY-101,[DB14803],1.0
2,0,Angiotensin 1-7,[DB11720],1.0
3,1,Antibiotic nos,[],0.0
4,1,Auxora,[],0.0
5,1,Auxora,[],0.0
6,0,AZD1222,[DB15656],1.0
7,0,Berberine,[DB04115],1.0
8,1,Beta blocker,[],0.0
9,1,Calcineurin inhibitors nos,[],0.0


(165, 4)

,NewDrug,Trail_Drug,Trail_Drug_DBID,iDBIDCount
0,0,Amlodipine,[DB00381],1.0
1,0,AMY-101,[DB14803],1.0
2,0,Angiotensin 1-7,[DB11720],1.0
6,0,AZD1222,[DB15656],1.0
7,0,Berberine,[DB04115],1.0
16,0,Eritoran,[DB04933],1.0
18,0,FX06,[DB05685],1.0
19,0,Gimsilumab,[DB15280],1.0
23,0,Hydroxychloroquine sulfate,[DB01611],1.0
24,0,Hydroxychloroquine Sulfate,[DB01611],1.0


(190, 4)

,NewDrug,Trail_Drug,Trail_Drug_DBID,iDBIDCount
11,1,Calcium,[DB01373],1.0
12,1,Cefixime,[DB00671],1.0
13,1,Chlorhexidine,[DB00878],1.0
15,1,Dextromethorphan,[DB00514],1.0
28,1,Loxoprofen,[DB09212],1.0
31,1,Mycophenolate mofetil,[DB00688],1.0
33,1,Norepinephrine,[DB00368],1.0
36,1,Remifentanil,[DB00899],1.0
38,1,Selenium,"[DB11127, DB11135]",2.0
41,1,Spironolactone,[DB00421],1.0


(178, 4)

,NewDrug,Trail_Drug,Trail_Drug_DBID,iDBIDCount
3,1,Antibiotic nos,[],0.0
4,1,Auxora,[],0.0
5,1,Auxora,[],0.0
8,1,Beta blocker,[],0.0
9,1,Calcineurin inhibitors nos,[],0.0
10,1,Calcineurin inhibitors nos,[],0.0
14,1,Darunavir/cobicistat,[],0.0
20,1,Hemoperfusion,[],0.0
26,1,Interferon kappa + TFF2,[],0.0
30,1,Mesenchymal stem cell transplant,[],0.0


In [11]:
truthMatch_df = truthFinal_df[truthFinal_df.iDBIDCount!=0]
#display(truthMatch_df.shape)
#display(truthMatch_df.head(40))
display(truthMatch_df['NewDrug'].value_counts())
display(truthMatch_df['iDBIDCount'].value_counts())

tmids = truthMatch_df.apply(lambda x: pd.Series(x['Trail_Drug_DBID']),axis=1).stack().reset_index(level=1, drop=True)
tmids.name = 'iDBID'
#display(tmids)

truthMatch_df = truthMatch_df.drop(['Trail_Drug_DBID', 'iDBIDCount'], axis=1).join(tmids)
display(truthMatch_df.shape)
display(truthMatch_df[10:30])


1    190
0    165
Name: NewDrug, dtype: int64

1.0    333
2.0     17
4.0      4
5.0      1
Name: iDBIDCount, dtype: int64

(388, 3)

,NewDrug,Trail_Drug,iDBID
18,0,FX06,DB05685
19,0,Gimsilumab,DB15280
23,0,Hydroxychloroquine sulfate,DB01611
24,0,Hydroxychloroquine Sulfate,DB01611
28,1,Loxoprofen,DB09212
29,0,Maraviroc,DB04835
31,1,Mycophenolate mofetil,DB00688
33,1,Norepinephrine,DB00368
36,1,Remifentanil,DB00899
37,0,rNAPc2,DB06552


In [12]:
truthMatch_df.drop_duplicates(subset='iDBID', keep='first', inplace=True)
display(truthMatch_df.shape)
display(truthMatch_df[10:30])
display(truthMatch_df['NewDrug'].value_counts())


(348, 3)

,NewDrug,Trail_Drug,iDBID
18,0,FX06,DB05685
19,0,Gimsilumab,DB15280
23,0,Hydroxychloroquine sulfate,DB01611
28,1,Loxoprofen,DB09212
29,0,Maraviroc,DB04835
31,1,Mycophenolate mofetil,DB00688
33,1,Norepinephrine,DB00368
36,1,Remifentanil,DB00899
37,0,rNAPc2,DB06552
38,1,Selenium,DB11127


1    193
0    155
Name: NewDrug, dtype: int64

In [13]:
# flag if DB ID has proteins associated or not
display(dbProteins_df.shape)
display(dbProteins_df.head())

# count proteins per DBID
proteinCount_df = pd.DataFrame(dbProteins_df.drugbank_id.value_counts().reset_index().values, columns=["iDBID", "ProteinCount"])
proteinCount_df.sort_values(by=['iDBID'], inplace=True)
proteinCount_df.reset_index(drop=True, inplace=True)
display(proteinCount_df.shape)
display(proteinCount_df.head())

display(truthMatch_df.shape)
display(truthMatch_df.head())

# merge count in
truthMatch2_df = truthMatch_df.merge(proteinCount_df, how="left", on='iDBID').fillna(0)
display(truthMatch2_df.shape)
display(truthMatch2_df.head())


(25940, 4)

,drugbank_id,category,uniprot_id,organism
0,DB00006,target,P00734,Humans
1,DB00006,enzyme,P05164,Humans
2,DB00007,target,P30968,Humans
3,DB00014,target,P22888,Humans
4,DB00014,target,P30968,Humans


(7469, 2)

,iDBID,ProteinCount
0,DB00006,2
1,DB00007,1
2,DB00014,2
3,DB00027,1
4,DB00035,5


(348, 3)

,NewDrug,Trail_Drug,iDBID
0,0,Amlodipine,DB00381
1,0,AMY-101,DB14803
2,0,Angiotensin 1-7,DB11720
6,0,AZD1222,DB15656
7,0,Berberine,DB04115


(348, 4)

,NewDrug,Trail_Drug,iDBID,ProteinCount
0,0,Amlodipine,DB00381,14
1,0,AMY-101,DB14803,0
2,0,Angiotensin 1-7,DB11720,0
3,0,AZD1222,DB15656,0
4,0,Berberine,DB04115,2


In [14]:
# merge fda truth in to validate new/old status

# first dedupe old Truth_df
display(Truth_df.shape)
Truth_df.drop_duplicates(subset='drugbank_id', keep='first', inplace=True)
display(Truth_df.shape)
display(Truth_df[['drugbank_id', 'truth']].head())

truthMatch2_df = truthMatch2_df.merge(Truth_df[['drugbank_id', 'truth']], how="left", left_on='iDBID', right_on='drugbank_id').fillna(0)
display(truthMatch2_df.shape)
display(truthMatch2_df.head())

display(pd.crosstab(truthMatch2_df.truth, truthMatch2_df.NewDrug, margins=True))

# fix incorrect NewDrug flags
truthMatch2_df.NewDrug[truthMatch2_df.truth==1] = 0
display(pd.crosstab(truthMatch2_df.truth, truthMatch2_df.NewDrug, margins=True))


(315, 6)

(286, 6)

,drugbank_id,truth
0,DB14803,1
1,DB15121,1
2,DB15656,1
3,DB01281,1
4,DB15327,1


(348, 6)

,NewDrug,Trail_Drug,iDBID,ProteinCount,drugbank_id,truth
0,0,Amlodipine,DB00381,14,DB00381,1.0
1,0,AMY-101,DB14803,0,DB14803,1.0
2,0,Angiotensin 1-7,DB11720,0,DB11720,1.0
3,0,AZD1222,DB15656,0,DB15656,1.0
4,0,Berberine,DB04115,2,DB04115,1.0


NewDrug,0,1,All
truth,,,
0.0,0,182,182
1.0,155,11,166
All,155,193,348


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


NewDrug,0,1,All
truth,,,
0.0,0,182,182
1.0,166,0,166
All,166,182,348


In [15]:
display(truthMatch2_df.shape)
display(truthMatch2_df.head())
display(pd.crosstab(truthMatch2_df.ProteinCount>0, truthMatch2_df.NewDrug, margins=True))


(348, 6)

,NewDrug,Trail_Drug,iDBID,ProteinCount,drugbank_id,truth
0,0,Amlodipine,DB00381,14,DB00381,1.0
1,0,AMY-101,DB14803,0,DB14803,1.0
2,0,Angiotensin 1-7,DB11720,0,DB11720,1.0
3,0,AZD1222,DB15656,0,DB15656,1.0
4,0,Berberine,DB04115,2,DB04115,1.0


NewDrug,0,1,All
ProteinCount,,,
False,66,27,93
True,100,155,255
All,166,182,348


In [16]:
# UniProt - protein IDs / synonyms  ( to merge in with STRING and Virus/Protein data so they can merge with DrugBank)

# exclude UniProt IDs beginning with A0A per this reference:
# https://ftp.uniprot.org/pub/databases/uniprot/knowledgebase/docs/sec%5Fac.txt

with open(UniProtPath + 'HUMAN_9606_idmapping.dat') as f:
    row_count = sum(1 for row in f)
display(row_count)

# can I read all of just column 1 ?
# if yes - I can use that to create an array of only STRING, only Gene_Name and only GeneID 
#    and then just read columns 0 & 2 from THAT subset
# else if that runs into space limits - just use GREP to extract the rows we want !
#    grep Gene_Name *.dat
altIDNames_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols = [1], sep='\t', header=None)
altIDNames_df.columns = ['altIDName']
display(altIDNames_df.shape)
display(altIDNames_df.head())
display(altIDNames_df.altIDName.value_counts())

STRINGSkip = altIDNames_df.index[altIDNames_df.altIDName != 'STRING']
EnsemblProSkip = altIDNames_df.index[altIDNames_df.altIDName != 'Ensembl_PRO']
GNameSkip = altIDNames_df.index[altIDNames_df.altIDName != 'Gene_Name']
GIDSkip = altIDNames_df.index[altIDNames_df.altIDName != 'GeneID']

del altIDNames_df


#-------------------
# STRING ID matching
display(STRINGSkip.shape)
UniProtSTRINGIDs_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(STRINGSkip), sep='\t', header=None)
UniProtSTRINGIDs_df.columns = ['UniProtID', 'STRINGID']
display(UniProtSTRINGIDs_df.shape)
display(UniProtSTRINGIDs_df.head())

#-------------------
# Ensembl Pro matching
display(EnsemblProSkip.shape)
UniProtEnsemblePro_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(EnsemblProSkip), sep='\t', header=None)
UniProtEnsemblePro_df.columns = ['UniProtID', 'Ensembl_PRO']
UniProtEnsemblePro_df['Ensembl_PRO9606'] = '9606.'+UniProtEnsemblePro_df['Ensembl_PRO']
display(UniProtEnsemblePro_df.shape)
display(UniProtEnsemblePro_df.head())

#-------------------
# Gene ID matching
display(GIDSkip.shape)
UniProtGeneIDs_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(GIDSkip), sep='\t', header=None)
UniProtGeneIDs_df.columns = ['UniProtID', 'GeneID']

# strip out duplicate UniProt GeneID rows - or should we be setting an order preference
UniProtGeneIDs_df['FirstChar'] = UniProtGeneIDs_df['UniProtID'].str.slice(0,1)
UniProtGeneIDs_df['A0A_Type'] = (UniProtGeneIDs_df['UniProtID'].str.slice(0,3) == 'A0A')
UniProtGeneIDs_df['ID_Length'] = UniProtGeneIDs_df['UniProtID'].str.len()
cols = ['FirstChar', 'A0A_Type', 'ID_Length']
display(UniProtGeneIDs_df.groupby(cols)[cols].count())

#UniProtGeneIDs_df = UniProtGeneIDs_df[UniProtGeneIDs_df['ID_Length'] == 6]
UniProtGeneIDs_df = UniProtGeneIDs_df[UniProtGeneIDs_df['A0A_Type'] == False]
#UniProtGeneIDs_df = UniProtGeneIDs_df[UniProtGeneIDs_df['FirstChar'].isin(['O', 'P', 'Q'])]
UniProtGeneIDs_df.drop(columns=['FirstChar', 'A0A_Type', 'ID_Length'], inplace=True)

# if > 1 UniProt row for a Host_Gene_ID, then keep P, then Q, then first of any remaining UniProt row(s)
GeneRows = UniProtGeneIDs_df.GeneID.value_counts()
print('The gene IDs with the top 25 most protein counts: ', GeneRows[:25])

display(UniProtGeneIDs_df.shape)
display(UniProtGeneIDs_df.head())

#-------------------
# Gene Name matching
display(GNameSkip.shape)
UniProtGeneNames_df = pd.read_csv(UniProtPath + 'HUMAN_9606_idmapping.dat', usecols=[0,2], skiprows=set(GNameSkip), sep='\t', header=None)
UniProtGeneNames_df.columns = ['UniProtID', 'GeneName']
display(UniProtGeneNames_df.shape)
display(UniProtGeneNames_df.head())

# will probably need to repeat gene-id cleansing steps above if using gene name to match




6050174

(6050174, 1)

,altIDName
0,UniProtKB-ID
1,Gene_Name
2,GI
3,GI
4,GI


EMBL                   993366
GI                     991294
EMBL-CDS               918458
UniParc                227071
CRC64                  194237
UniRef100              194237
UniProtKB-ID           194237
NCBI_TaxID             194237
UniRef90               193817
UniRef50               193817
Gene_Name              158688
ChiTaRS                127284
Ensembl_PRO            110171
Ensembl_TRS            110171
RefSeq_NT               80596
RefSeq                  80596
Ensembl                 79957
ProteomicsDB            76304
HGNC                    74980
GeneTree                71634
EuPathDB                69202
UCSC                    64524
PDB                     63958
HOGENOM                 50988
Reactome                41059
GeneID                  32993
GenomeRNAi              32087
CCDS                    29904
KEGG                    25852
DrugBank                25550
                        ...  
eggNOG                  19458
STRING                  18882
TreeFam   

(6031292,)

...

(27978, 2)

,UniProtID,GeneID
0,P31946,7529
1,P62258,7531
2,Q04917,7533
3,P61981,7532
4,P31947,2810


(5891486,)

(158688, 2)

,UniProtID,GeneName
0,P31946,YWHAB
1,P62258,YWHAE
2,Q04917,YWHAH
3,P61981,YWHAG
4,P31947,SFN


In [17]:
# example of a gene with 29 proteins (4 of the A0A type)
# https://worldwide.promega.com/FindMyGene/GeneDetail.aspx?ncbiid=7404
display(UniProtGeneIDs_df[UniProtGeneIDs_df.GeneID == 7404])


,UniProtID,GeneID
17945,O14607,7404
24304,E1NZ93,7404
25361,E1U1M9,7404
25372,F5GWV3,7404
25670,F4MH46,7404
26032,F4MH32,7404
26322,F4MH35,7404
26621,F4MH27,7404
27330,F4MH47,7404
27440,F5H3N7,7404


In [18]:
# VirusHost - virus / protein
VirusHostProt_df = pd.read_csv(VHProtPath + 'HCoV-associated host proteins.csv')
display(VirusHostProt_df.shape)
display(VirusHostProt_df.head())

# now get UniProtID merged in on Gene_ID (match on Host_Protein should be equivalent)
#display(UniProtGeneIDs_df.head())

VirusHostProt_df = pd.merge(VirusHostProt_df, UniProtGeneIDs_df, how="left", left_on='Host_Gene_ID', right_on='GeneID', indicator=True)
print("after match (expecting both=130, right/left only = 0): ")
display(VirusHostProt_df['_merge'].value_counts())
VirusHostProt_df.drop(columns=['_merge', 'GeneID'], inplace=True)
display(VirusHostProt_df.shape)
display(VirusHostProt_df.head())


(130, 4)

,Coronavirus,Host_Protein,Host_Gene_ID,PubMed_ID
0,IBV,NONO,4841,23637410
1,IBV,GSK3A,2931,23637410
2,IBV,GSK3B,2932,23637410
3,IBV,PABPC1,26986,23637410
4,IBV,PABPC4,8761,23637410


after match (expecting both=130, right/left only = 0): 


both          213
right_only      0
left_only       0
Name: _merge, dtype: int64

(213, 5)

,Coronavirus,Host_Protein,Host_Gene_ID,PubMed_ID,UniProtID
0,IBV,NONO,4841,23637410,Q15233
1,IBV,GSK3A,2931,23637410,P49840
2,IBV,GSK3B,2932,23637410,P49841
3,IBV,GSK3B,2932,23637410,Q6FI27
4,IBV,PABPC1,26986,23637410,P11940


In [19]:
#display(VirusHostProt_df.iloc[[0,1,2,4,5]])
display(VirusHostProt_df.iloc[:200])


,Coronavirus,Host_Protein,Host_Gene_ID,PubMed_ID,UniProtID
0,IBV,NONO,4841,23637410,Q15233
1,IBV,GSK3A,2931,23637410,P49840
2,IBV,GSK3B,2932,23637410,P49841
3,IBV,GSK3B,2932,23637410,Q6FI27
4,IBV,PABPC1,26986,23637410,P11940
5,IBV,PABPC4,8761,23637410,Q13310
6,IBV,HNRNPA1,3178,23637410,P09651
7,IBV,HNRNPA2B1,3181,23637410,P22626
8,IBV,NPM1,4869,23637410,P06748
9,IBV,G3BP1,10146,23637410,Q13283


<b>Duplicates exist even after elminating A0A type matches</b><br>
<br>
one example is gene_id 2932 = P49841 and Q6FI27<br>
<br>
https://www.uniprot.org/uniprot/?query=yourlist:M202103028471C63D39733769F8E060B506551E12176354Y&sort=yourlist:M202103028471C63D39733769F8E060B506551E12176354Y&columns=yourlist(M202103028471C63D39733769F8E060B506551E12176354Y),id,entry%20name,reviewed,protein%20names,genes,organism,length


In [20]:
# STRING - protein/protein

# NOTE - each pair is in the file 2x so only UniProtID1 needs to be matched
# Example below:
#[sasdemo@sasserver STRING]$ grep ENSP00000000233 *.txt | grep -e ENSP00000272298
#9606.ENSP00000000233 9606.ENSP00000272298 490
#9606.ENSP00000272298 9606.ENSP00000000233 490

stringPP_df = pd.read_csv(STRINGPath + '9606.protein.links.v11.0.txt', sep=' ')
display(stringPP_df.shape)
display(stringPP_df.head())

# merge Protein 1 to STRING on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtSTRINGIDs_df, how="left", left_on='protein1', right_on='STRINGID', indicator=True)
#display(stringPP_df['_merge'].value_counts())
stringPP_df.rename(columns = {'UniProtID':'UniProtID1', 'STRINGID':'STRINGID1'}, inplace = True)
stringPP_df.drop(columns=['_merge'], inplace=True)

# merge Protein 1 to Ensembl_PRO on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtEnsemblePro_df[['UniProtID', 'Ensembl_PRO9606']], how="left", left_on='protein1', right_on='Ensembl_PRO9606', indicator=True)
#display(stringPP_df['_merge'].value_counts())

# coalesce Protein 1 STRING and Ensemble_PRO columns (use Ensembl_PRO if STRING is missing)
stringPP_df['UniProtID1'] = stringPP_df['UniProtID1'].mask(pd.isnull, stringPP_df['UniProtID'])
stringPP_df.rename(columns = {'Ensembl_PRO9606':'Ensembl_PRO96061'}, inplace = True)
stringPP_df.drop(columns=['_merge', 'UniProtID'], inplace=True)

# merge protein 2 to STRING on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtSTRINGIDs_df, how="left", left_on='protein2', right_on='STRINGID', indicator=True)
#display(stringPP_df['_merge'].value_counts())
stringPP_df.rename(columns = {'UniProtID':'UniProtID2', 'STRINGID':'STRINGID2'}, inplace = True)
stringPP_df.drop(columns=['_merge'], inplace=True)

# merge Protein 2 to Ensembl_PRO on UniProt .dat file
stringPP_df = pd.merge(stringPP_df, UniProtEnsemblePro_df[['UniProtID', 'Ensembl_PRO9606']], how="left", left_on='protein2', right_on='Ensembl_PRO9606', indicator=True)

# coalesce Protein 2 STRING and Ensemble_PRO columns (use Ensembl_PRO if STRING is missing)
stringPP_df['UniProtID2'] = stringPP_df['UniProtID2'].mask(pd.isnull, stringPP_df['UniProtID'])
stringPP_df.rename(columns = {'Ensembl_PRO9606':'Ensembl_PRO96062'}, inplace = True)
stringPP_df.drop(columns=['_merge', 'UniProtID'], inplace=True)

print()
print('After matching to STRING and Ensembl_PRO in UniProt')
display(stringPP_df.shape)
print("Rows with no match for STRING: ", stringPP_df.STRINGID1.isnull().count())
print("Rows with no match for Ensebml_PRO: ", stringPP_df.Ensembl_PRO96061.isnull().count())
print("Rows with no match for either: ", stringPP_df.UniProtID1.isnull().count())
#print("Rows with no match for either: ", stringPP_df.UniProtID1.isnull().value_counts())
print("NOTE: next most likely reason for missing is depricated IDs.  see note in markup cell below")
print()
display(stringPP_df.head())

print()
print('Sample of Null matches:')
display(stringPP_df[stringPP_df.UniProtID1.isnull()].head())
#display(stringPP_df[stringPP_df.UniProtID1.isnull()].groupby('protein1')['protein1'].count())
print("unique IDs with no match: ", stringPP_df[stringPP_df.UniProtID1.isnull()].groupby('protein1')['protein1'].count().count())

# reset dataframe to contain only relevant columns
stringPP_df = stringPP_df[['protein1', 'UniProtID1', 'protein2', 'UniProtID2', 'combined_score']]

(11759454, 3)

,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000272298,490
1,9606.ENSP00000000233,9606.ENSP00000253401,198
2,9606.ENSP00000000233,9606.ENSP00000401445,159
3,9606.ENSP00000000233,9606.ENSP00000418915,606
4,9606.ENSP00000000233,9606.ENSP00000327801,167



After matching to STRING and Ensembl_PRO in UniProt


(11768350, 9)

Rows with no match for STRING:  11768350
Rows with no match for Ensebml_PRO:  11768350
Rows with no match for either:  11768350
NOTE: next most likely reason for missing is depricated IDs.  see note in markup cell below



,protein1,protein2,combined_score,UniProtID1,STRINGID1,Ensembl_PRO96061,UniProtID2,STRINGID2,Ensembl_PRO96062
0,9606.ENSP00000000233,9606.ENSP00000272298,490,P84085,9606.ENSP00000000233,9606.ENSP00000000233,P0DP24,NaN,9606.ENSP00000272298
1,9606.ENSP00000000233,9606.ENSP00000253401,198,P84085,9606.ENSP00000000233,9606.ENSP00000000233,O43307,9606.ENSP00000253401,9606.ENSP00000253401
2,9606.ENSP00000000233,9606.ENSP00000401445,159,P84085,9606.ENSP00000000233,9606.ENSP00000000233,O75460,9606.ENSP00000401445,9606.ENSP00000401445
3,9606.ENSP00000000233,9606.ENSP00000418915,606,P84085,9606.ENSP00000000233,9606.ENSP00000000233,P42771,9606.ENSP00000418915,9606.ENSP00000418915
4,9606.ENSP00000000233,9606.ENSP00000327801,167,P84085,9606.ENSP00000000233,9606.ENSP00000000233,P07237,9606.ENSP00000327801,9606.ENSP00000327801



Sample of Null matches:


,protein1,protein2,combined_score,UniProtID1,STRINGID1,Ensembl_PRO96061,UniProtID2,STRINGID2,Ensembl_PRO96062
144960,9606.ENSP00000195455,9606.ENSP00000384273,181,NaN,NaN,NaN,Q04206,9606.ENSP00000384273,9606.ENSP00000384273
144961,9606.ENSP00000195455,9606.ENSP00000308057,242,NaN,NaN,NaN,Q16143,9606.ENSP00000308057,9606.ENSP00000308057
144962,9606.ENSP00000195455,9606.ENSP00000355812,181,NaN,NaN,NaN,O95684,9606.ENSP00000355812,9606.ENSP00000355812
144963,9606.ENSP00000195455,9606.ENSP00000358770,398,NaN,NaN,NaN,Q9H2G2,9606.ENSP00000358770,9606.ENSP00000358770
144964,9606.ENSP00000195455,9606.ENSP00000192788,175,NaN,NaN,NaN,Q6BDS2,9606.ENSP00000192788,9606.ENSP00000192788


unique IDs with no match:  99


<b>searching UniProt ID Mapping.dat file for unmatched STRING IDs(without 9606 prefix
gives no STRING IDs, but Ensembl_PRO</b>

[sasdemo@sasserver UniProt]$ ls -ltr<br>
-rw-r--r-- 1 cas sas 159854967 Jan 26 14:20 HUMAN_9606_idmapping.dat<br>

[sasdemo@sasserver UniProt]$ grep -e "ENSP00000016946" *.dat<br>
Q99666-1        Ensembl_PRO     ENSP00000016946<br>

[sasdemo@sasserver UniProt]$ grep -e "ENSP00000244537" *.dat<br>
P62805  Ensembl_PRO     ENSP00000244537<br>

[sasdemo@sasserver UniProt]$ grep -e "ENSP00000244537" *.dat<br>
P62805  Ensembl_PRO     ENSP00000244537<br>

<b>searching UniProt ID Mapping.dat for a matched STRINGID (without 9606 prefix)
gives both</b>
[sasdemo@sasserver UniProt]$ grep -e "ENSP00000380178" *.dat<br>
P62253  STRING  9606.ENSP00000380178<br>
P62253  Ensembl_PRO     ENSP00000380178<br>

<b><i>conclusion --> strip out 9606. and try matching to enesmbl_pro.  if that does not work, match STRING first, then match missing to Ensembl_PRO</b></i>


<b>ENSP #s that do not match either STRING or Ensembl_PRO may be depricated IDs such as this one:</b><br>
9606.ENSP00000195455 --> ENST00000195455<br>
9606.ENSP00000377806 --> ENST00000377806<br>
<br>
https://useast.ensembl.org/Homo_sapiens/Transcript/Idhistory/Protein?t=ENSP00000195455
https://useast.ensembl.org/Homo_sapiens/Transcript/Idhistory/Protein?t=ENSP00000377806


## 3) load base to CAS

In [21]:
if casenabled == True:
    #s.fileinfo(caslib='public') # files have been saved to caslib table backing store
    s.tableinfo(caslib='public') # tables are loaded in memory (and may not have been saved)


In [22]:
# NOTE: upload_file() can upload disk files into CAS without using local dataframe memory


In [23]:
# UniProt - select rows for matching:

if casenabled == True:
    s.droptable(caslib='public', name='UniProtGeneIDs', quiet=True)
    UniProtGeneIDs_ct = s.upload_file(data=UniProtGeneIDs_df, casout=dict(name='UniProtGeneIDs', caslib='public', replication=0, promote=True))
    display(UniProtGeneIDs_ct.info())
    display(UniProtGeneIDs_ct.head())

    s.droptable(caslib='public', name='UniProtSTRINGIDs', quiet=True)
    UniProtSTRINGIDs_ct = s.upload_file(data=UniProtSTRINGIDs_df, casout=dict(name='UniProtSTRINGIDs', caslib='public', replication=0, promote=True))
    UniProtSTRINGIDs_ct.info()
    display(UniProtSTRINGIDs_ct.head())


    s.droptable(caslib='public', name='UniProtGeneNames', quiet=True)
    UniProtGeneNames_ct = s.upload_file(data=UniProtGeneNames_df, casout=dict(name='UniProtGeneNames', caslib='public', replication=0, promote=True))
    UniProtGeneNames_ct.info()
    display(UniProtGeneNames_ct.head())

UniProtGeneIDs_df.to_csv(CSVOut+"/"+"UniProtGeneIDs.csv")
UniProtSTRINGIDs_df.to_csv(CSVOut+"/"+"UniProtSTRINGIDs.csv")
UniProtGeneNames_df.to_csv(CSVOut+"/"+"UniProtGeneNames.csv")


NOTE: Cloud Analytic Services made the uploaded file available as table UNIPROTGENEIDS in caslib public.
NOTE: The table UNIPROTGENEIDS has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('UNIPROTGENEIDS', caslib='public')
Data columns (total 2 columns):
               N   Miss     Type
UniProtID  27978  False  varchar
GeneID     27978  False   double
dtypes: double(1), varchar(1)
data size: 839340
vardata size: 167868
memory usage: 0


None

,UniProtID,GeneID
0,P31946,7529.0
1,P62258,7531.0
2,Q04917,7533.0
3,P61981,7532.0
4,P31947,2810.0


NOTE: Cloud Analytic Services made the uploaded file available as table UNIPROTSTRINGIDS in caslib public.
NOTE: The table UNIPROTSTRINGIDS has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('UNIPROTSTRINGIDS', caslib='public')
Data columns (total 2 columns):
               N   Miss     Type
UniProtID  18882  False  varchar
STRINGID   18882  False  varchar
dtypes: varchar(2)
data size: 1095556
vardata size: 491332
memory usage: 0


,UniProtID,STRINGID
0,P31946,9606.ENSP00000361930
1,P62258,9606.ENSP00000264335
2,Q04917,9606.ENSP00000248975
3,P61981,9606.ENSP00000306330
4,P31947,9606.ENSP00000340989


NOTE: Cloud Analytic Services made the uploaded file available as table UNIPROTGENENAMES in caslib public.
NOTE: The table UNIPROTGENENAMES has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('UNIPROTGENENAMES', caslib='public')
Data columns (total 2 columns):
                N   Miss     Type
UniProtID  158688  False  varchar
GeneName   158688  False  varchar
dtypes: varchar(2)
data size: 7099339
vardata size: 2021323
memory usage: 0


,UniProtID,GeneName
0,P31946,YWHAB
1,P62258,YWHAE
2,Q04917,YWHAH
3,P61981,YWHAG
4,P31947,SFN


In [24]:
# ClinicalTrials Truth (could possibly benefit from more cleaning):
Truth_df.to_csv(CSVOut+"/"+"Truth.csv")
if casenabled == True:
    s.droptable(caslib='public', name='Truth', quiet=True)
    Truth_ct = s.upload_file(data=Truth_df, casout=dict(name='Truth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))
    Truth_ct.info()
    display(Truth_ct.head())

# truthMatch2_df has information on the "new" source (non FDA Clinical Trials)
truthMatch2_df.to_csv(CSVOut+"/"+"newTruth.csv")
if casenabled == True:
    s.droptable(caslib='public', name='newTruth', quiet=True)
    newTruth_ct = s.upload_file(data=truthMatch2_df, casout=dict(name='newTruth', caslib='public', replication=0, promote=True, label="Clinical Trials Truth: NOTE needs to be refined"))
    newTruth_ct.info()
    display(newTruth_ct.head())


NOTE: Cloud Analytic Services made the uploaded file available as table TRUTH in caslib public.
NOTE: The table TRUTH has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('TRUTH', caslib='public')
Data columns (total 6 columns):
                 N   Miss     Type
drugbank_id    286  False  varchar
truth          286  False   double
iDBIDCount     286  False   double
iName          286  False  varchar
inputRowCount  286  False   double
inputRowIDs    286  False  varchar
dtypes: double(3), varchar(3)
data size: 29307
vardata size: 8715
memory usage: 0


,drugbank_id,truth,iDBIDCount,iName,inputRowCount,inputRowIDs
0,DB14803,1.0,1.0,AMY-101,1.0,1891
1,DB15121,1.0,1.0,AT-001,1.0,308
2,DB15656,1.0,1.0,AZD1222,3.0,[ 14 19 231]
3,DB01281,1.0,1.0,Abatacept,3.0,[309 583 607]
4,DB15327,1.0,1.0,Abivertinib,1.0,1352


NOTE: Cloud Analytic Services made the uploaded file available as table NEWTRUTH in caslib public.
NOTE: The table NEWTRUTH has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('NEWTRUTH', caslib='public')
Data columns (total 6 columns):
                N   Miss     Type
NewDrug       348  False   double
Trail_Drug    348  False  varchar
iDBID         348  False  varchar
ProteinCount  348  False   double
drugbank_id   348  False  varchar
truth         348  False   double
dtypes: double(3), varchar(3)
data size: 32685
vardata size: 7629
memory usage: 0


,NewDrug,Trail_Drug,iDBID,ProteinCount,drugbank_id,truth
0,0.0,Amlodipine,DB00381,14.0,DB00381,1.0
1,0.0,AMY-101,DB14803,0.0,DB14803,1.0
2,0.0,Angiotensin 1-7,DB11720,0.0,DB11720,1.0
3,0.0,AZD1222,DB15656,0.0,DB15656,1.0
4,0.0,Berberine,DB04115,2.0,DB04115,1.0


In [25]:
# VirusHost Proteins
if casenabled == True:
    s.droptable(caslib='public', name='VirusHostProt', quiet=True)
    VirusHostProt_ct = s.upload_file(data=VirusHostProt_df, casout=dict(name='VirusHostProt', caslib='public', replication=0, promote=True, label="Virus Host Protein Relationships NOTE: Many Proteins had 1+ UniProt IDs"))
    display(VirusHostProt_ct.info())
    display(VirusHostProt_ct.head())

VirusHostProt_df.to_csv(CSVOut+"/"+"VirusHostProt.csv")


NOTE: Cloud Analytic Services made the uploaded file available as table VIRUSHOSTPROT in caslib public.
NOTE: The table VIRUSHOSTPROT has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('VIRUSHOSTPROT', caslib='public')
Data columns (total 5 columns):
                N   Miss     Type
Coronavirus   213  False  varchar
Host_Protein  213  False  varchar
Host_Gene_ID  213  False   double
PubMed_ID     213  False  varchar
UniProtID     213  False  varchar
dtypes: double(1), varchar(4)
data size: 20155
vardata size: 4819
memory usage: 0


None

,Coronavirus,Host_Protein,Host_Gene_ID,PubMed_ID,UniProtID
0,IBV,NONO,4841.0,23637410,Q15233
1,IBV,GSK3A,2931.0,23637410,P49840
2,IBV,GSK3B,2932.0,23637410,P49841
3,IBV,GSK3B,2932.0,23637410,Q6FI27
4,IBV,PABPC1,26986.0,23637410,P11940


In [26]:
# STRING - protein protein data
if casenabled == True:
    s.droptable(caslib='public', name='stringPP', quiet=True)
    stringPP_ct = s.upload_file(data=stringPP_df, casout=dict(name='stringPP', caslib='public', replication=0, promote=True, label="STRING Protein Protein interactions NOTE Missing UniProt Matches"))
    stringPP_ct.info()
    display(stringPP_ct.head())

stringPP_df.to_csv(CSVOut+"/"+"stringPP.csv")


NOTE: Cloud Analytic Services made the uploaded file available as table STRINGPP in caslib public.
NOTE: The table STRINGPP has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('STRINGPP', caslib='public')
Data columns (total 5 columns):
                       N   Miss     Type
protein1        11768350  False  varchar
UniProtID1      11747277   True  varchar
protein2        11768350  False  varchar
UniProtID2      11747277   True  varchar
combined_score  11768350  False   double
dtypes: double(1), varchar(4)
data size: 1459580092
vardata size: 612258892
memory usage: 0


,protein1,UniProtID1,protein2,UniProtID2,combined_score
0,9606.ENSP00000064778,Q92567,9606.ENSP00000411848,Q5T0T0,261.0
1,9606.ENSP00000064778,Q92567,9606.ENSP00000306983,Q9ULM3,473.0
2,9606.ENSP00000064778,Q92567,9606.ENSP00000446479,Q9BVA6,241.0
3,9606.ENSP00000064778,Q92567,9606.ENSP00000344424,Q92537,627.0
4,9606.ENSP00000064778,Q92567,9606.ENSP00000378972,P50225,152.0


In [27]:
#NOTE: indication and description cause data errors when loading to a CASLIB - figure it out later

display(dbSmallM_df[:5])

# clean up drug bank small molecule list and merge in clinical trials "truth" flag
x = dbSmallM_df[['drugbank_id', 'name', 'inchi', 'groups', 'atc_codes', 'categories']]

# replace truth_df with truthMatch2_df
#x = pd.merge(x, Truth_df[['drugbank_id', 'truth']], on='drugbank_id', how='left', indicator=True)
newTruth = truthMatch2_df[['iDBID', 'NewDrug', 'ProteinCount']]
newTruth['truth'] = 1
display(newTruth.head()) 

x = pd.merge(x, newTruth, left_on='drugbank_id', right_on='iDBID', how='left', indicator=True)

display(x['_merge'].value_counts())
x.drop(columns=['_merge'], inplace=True)

x.truth = x.truth.fillna(0)
display(x['truth'].value_counts())


# DrugBank Tables
if casenabled == True:
    s.droptable(caslib='public', name='dbSmallM_Truth', quiet=True)
    #dbSmallM_ct = s.upload_file(data=dbSmallM_df, casout=dict(name='dbSmallM', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molecule drugs (NOTE additional attributes available plus stings need to be parsed)"))
    dbSmallM_ct = s.upload_file(data=x, casout=dict(name='dbSmallM_Truth', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molecule drugs (NOTE additional attributes available plus stings need to be parsed)"))
    dbSmallM_ct.info()
    display(dbSmallM_ct.shape)
    display(dbSmallM_ct.head())

x.to_csv(CSVOut+"/"+"dbSmallM_Truth.csv")


,drugbank_id,name,type,groups,atc_codes,categories,inchikey,inchi,description,indication
0,DB00006,Bivalirudin,small molecule,approved|investigational,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",OIRCOABEOLEUMC-GEJPAHFPSA-N,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,Bivalirudin is a synthetic 20 residue peptide ...,For treatment of heparin-induced thrombocytope...
1,DB00007,Leuprolide,small molecule,approved|investigational,L02AE51|L02AE02,Adrenal Cortex Hormones|Agents Causing Muscle ...,GFIJNRVAKGFPGQ-LIJARHBVSA-N,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,Leuprolide is a synthetic 9-residue peptide an...,Leuprolide is indicated for the palliative tre...
2,DB00014,Goserelin,small molecule,approved,L02AE03,"Adrenal Cortex Hormones|Amino Acids, Peptides,...",BLCLNMBMMGCOAS-URPVMXJPSA-N,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,"Goserelin is a synthetic hormone. In men, it s...",Goserelin is indicated for:\r\n\r\n- Use in co...
3,DB00027,Gramicidin D,small molecule,approved,R02AB30,"Amino Acids, Peptides, and Proteins|Anti-Bacte...",NDAYQJDHGXTBJL-MWWSRJDJSA-N,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,Gramcidin D is a heterogeneous mixture of thre...,"For treatment of skin lesions, surface wounds ..."
4,DB00035,Desmopressin,small molecule,approved,H01BA02,"Agents that produce hypertension|Amino Acids, ...",NFLWUMRGJYTJIN-PNIOQBSNSA-N,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,"Desmopressin (dDAVP), a synthetic analogue of ...",- Indicated for the treatment of nocturia due ...


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,iDBID,NewDrug,ProteinCount,truth
0,DB00381,0,14,1
1,DB14803,0,0,1
2,DB11720,0,0,1
3,DB15656,0,0,1
4,DB04115,0,2,1


left_only     11113
both            301
right_only        0
Name: _merge, dtype: int64

0.0    11113
1.0      301
Name: truth, dtype: int64

NOTE: Cloud Analytic Services made the uploaded file available as table DBSMALLM_TRUTH in caslib public.
NOTE: The table DBSMALLM_TRUTH has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('DBSMALLM_TRUTH', caslib='public')
Data columns (total 10 columns):
                  N   Miss     Type
drugbank_id   11414  False  varchar
name          11414  False  varchar
inchi         10765   True  varchar
groups        11414  False  varchar
atc_codes      2865   True  varchar
categories     6685   True  varchar
iDBID           301   True  varchar
NewDrug         301   True   double
ProteinCount    301   True   double
truth         11414  False   double
dtypes: double(3), varchar(7)
data size: 5593445
vardata size: 4041141
memory usage: 0


(11414, 10)

,drugbank_id,name,inchi,groups,atc_codes,categories,iDBID,NewDrug,ProteinCount,truth
0,DB00006,Bivalirudin,InChI=1S/C98H138N24O33/c1-5-52(4)82(96(153)122...,approved|investigational,B01AE06,"Amino Acids, Peptides, and Proteins|Anticoagul...",DB00006,1.0,2.0,1.0
1,DB00007,Leuprolide,InChI=1S/C59H84N16O12/c1-6-63-57(86)48-14-10-2...,approved|investigational,L02AE51|L02AE02,Adrenal Cortex Hormones|Agents Causing Muscle ...,,NaN,NaN,0.0
2,DB00014,Goserelin,InChI=1S/C59H84N18O14/c1-31(2)22-40(49(82)68-3...,approved,L02AE03,"Adrenal Cortex Hormones|Amino Acids, Peptides,...",,NaN,NaN,0.0
3,DB00027,Gramicidin D,InChI=1S/C96H135N19O16/c1-50(2)36-71(105-79(11...,approved,R02AB30,"Amino Acids, Peptides, and Proteins|Anti-Bacte...",,NaN,NaN,0.0
4,DB00035,Desmopressin,InChI=1S/C46H64N14O12S2/c47-35(62)15-14-29-40(...,approved,H01BA02,"Agents that produce hypertension|Amino Acids, ...",,NaN,NaN,0.0


In [28]:

# drug bank - drug/protein table(s)
if casenabled == True:
    s.droptable(caslib='public', name='dbProteins', quiet=True)
    dbProteins_ct = s.upload_file(data=dbProteins_df, casout=dict(name='dbProteins', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molcule related proteins (NOTE not all captured by XML parser yet)"))
    #dbProteins_ct = s.upload_file(data=dbProteins_df[:5], casout=dict(name='dbProteins', caslib='public', replication=0, promote=True, label="Drug Bank: Small Molcule related proteins"))
    dbProteins_ct.info()
    display(dbProteins_ct.shape)
    display(dbProteins_ct.head())

dbProteins_df.to_csv(CSVOut+"/"+"dbProteins.csv")


NOTE: Cloud Analytic Services made the uploaded file available as table DBPROTEINS in caslib public.
NOTE: The table DBPROTEINS has been created in caslib public from binary data uploaded to Cloud Analytic Services.
CASTable('DBPROTEINS', caslib='public')
Data columns (total 4 columns):
                 N   Miss     Type
drugbank_id  25940  False  varchar
category     25940  False  varchar
uniprot_id   25940  False  varchar
organism     25790   True  varchar
dtypes: varchar(4)
data size: 2434780
vardata size: 774620
memory usage: 0


(25940, 4)

,drugbank_id,category,uniprot_id,organism
0,DB00006,target,P00734,Humans
1,DB00006,enzyme,P05164,Humans
2,DB00007,target,P30968,Humans
3,DB00014,target,P22888,Humans
4,DB00014,target,P30968,Humans


3) Join tables and write result to CAS

In [29]:
#s.close()